<a href="https://colab.research.google.com/github/k1151msarandega/1st-order/blob/main/QC_Module_4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?id=1eBlhnvWo94RnzPK-86F5MzfFd86Cjb9C" width="65">


Created by: The AVELA Team
# **0. Welcome to the <a style="text-decoration:none;" href="https://towardsdatascience.com/cheat-sheet-for-google-colab-63853778c093"><font color='blue'>G</font><font color='red'>o</font><font color='Goldenrod'>o</font><font color='blue'>g</font><font color='green'>l</font><font color='red'>e</font><font color="black"> Colab</font></a> notebook!**
Make sure to read <u>every</u> cell in this notebook to get your Quantum Computing [Python](https://docs.google.com/document/d/1gen8uhv7UC_Qo5wT5paX8tesrSffhXlK9WDYunmcWgs/edit?usp=sharing) code working! For more guidance, you can check out the [python documentation](https://docs.python.org/3/tutorial/index.html) to learn more python and the [qiskit documentation](https://docs.quantum.ibm.com/). If you do not know how to code in at all here is a curated [AVELA Python Basics Course](https://tinyurl.com/AVELA-python-basics) to start with. \\
To run code, all you have to do is click the *run* button ▶️ (triangle in a circle). \\


**Run and analyze the provided example systems and pick a project. Create your own method or derive a direction from it or think of a new problem and come up with the solution.** \\


# ⚙️ Installation: Run the cell below first to install required libraries

In [ ]:
!pip install qiskit qiskit-aer pylatexenc

# Choose Project Based On The Categories Below

# 🔬 Quantum Technology Simulations

### These are science-focused simulations that demonstrate real quantum technologies and research use-cases, ideal for exploring curiosity about physics applications and performance analysis.

### 🔐 Quantum Secure Messaging App Simulation  


**Concepts:** BB84 QKD · Quantum Teleportation · Eavesdropper Detection

**Learn:**
- How secure keys are established.  
- How teleportation sends encrypted bits.  
- How to detect and react to eavesdropping.

**Explore:**  
Run the simulation, analyze key agreement, and teleport a quantum-encrypted message.

In [ ]:
# Quantum Secure Messaging App Simulation
# Combines BB84 Quantum Key Distribution (QKD), Quantum Teleportation, and Eavesdropping Detection

import numpy as np  # Used for generating random numbers and arrays
from qiskit import QuantumCircuit, transpile  # Tools for creating and compiling quantum circuits
from qiskit_aer import Aer  # Qiskit Aer simulator backend
from qiskit.visualization import plot_bloch_multivector  # For optional visualization
from qiskit.quantum_info import random_statevector, Statevector  # Tools for quantum state handling
import matplotlib.pyplot as plt  # For plotting (not used directly here)
import time  # Used to add delays for readability in output

# Settings
n_bits = 16  # Number of bits to generate in the BB84 protocol
np.random.seed(42)  # Seed the random number generator for reproducibility

# Simulates the BB84 quantum key distribution protocol
def generate_bb84_key(n, eve_present=False):
    simulator = Aer.get_backend('aer_simulator')  # Load Qiskit simulator
    alice_bits = np.random.randint(2, size=n)      # Alice randomly generates bits (0 or 1)
    alice_bases = np.random.randint(2, size=n)     # Alice randomly selects encoding bases (0=Z, 1=H)
    bob_bases = np.random.randint(2, size=n)       # Bob randomly selects measurement bases
    bob_results = []                               # Bob’s results after measurement
    eve_actions = []  # Stores Eve's basis choices (if present) for debugging

    print("\n[BB84] Starting quantum key distribution...")
    for i in range(n):
        print(f"  Round {i+1}/{n}:")
        qc = QuantumCircuit(1, 1)  # One-qubit, one-classical-bit circuit

        # Alice encodes her bit
        if alice_bits[i] == 1:
            qc.x(0)  # Flip qubit to |1⟩
            print("    Alice encodes bit: 1")
        else:
            print("    Alice encodes bit: 0")

        # Apply encoding basis
        if alice_bases[i] == 1:
            qc.h(0)  # Apply Hadamard to switch to X-basis
            print("    Alice uses Hadamard basis")
        else:
            print("    Alice uses Z basis")

        # Eve intercepts and measures qubit if present
        if eve_present:
            eve_basis = np.random.randint(2)  # Eve picks random basis
            eve_actions.append(eve_basis)
            print(f"    Eve intercepts using {'Hadamard' if eve_basis else 'Z'} basis")
            if eve_basis == 1:
                qc.h(0)
            qc.measure_all()  # Eve measures the qubit
            compiled = transpile(qc, simulator)
            result = simulator.run(compiled, shots=1, memory=True).result()
            bit_string = result.get_memory()[0]
            bit = int(bit_string.replace(' ', '')[0])  # Eve's result

            # Eve prepares new qubit based on her result
            qc = QuantumCircuit(1, 1)
            if bit == 1:
                qc.x(0)
            if eve_basis == 1:
                qc.h(0)
        else:
            eve_actions.append(None)

        # Bob applies his basis before measurement
        if bob_bases[i] == 1:
            qc.h(0)
            print("    Bob uses Hadamard basis")
        else:
            print("    Bob uses Z basis")

        # Bob measures the qubit
        qc.measure(0, 0)
        compiled = transpile(qc, simulator)
        result = simulator.run(compiled, shots=1).result()
        counts = result.get_counts()
        bob_bit = int(list(counts.keys())[0].replace(' ', ''))
        bob_results.append(bob_bit)
        print(f"    Bob measures: {bob_bit}\n")
        time.sleep(0.3)  # Delay for visualization

    # Sifting: Alice and Bob compare bases and keep matching bits
    sifted_key = []
    matched_indices = []
    for i in range(n):
        if alice_bases[i] == bob_bases[i]:
            sifted_key.append(bob_results[i])
            matched_indices.append(i)

    alice_sifted = [alice_bits[i] for i in matched_indices]
    return alice_sifted, sifted_key, matched_indices, eve_actions

# Randomly samples bits to detect discrepancies caused by eavesdropping
def simulate_eve_detection(alice_key, bob_key):
    sample_size = min(10, len(alice_key))  # Check up to 10 bits
    sample_indices = np.random.choice(len(alice_key), sample_size, replace=False)
    discrepancies = sum([alice_key[i] != bob_key[i] for i in sample_indices])
    return discrepancies / sample_size  # Return error rate

# Simulates quantum teleportation using a one-time pad key bit
def quantum_teleportation(message_bit, key_bit):
    simulator = Aer.get_backend('aer_simulator')
    qc = QuantumCircuit(3, 3)  # Qubit 0: message, 1 & 2: entangled pair

    # Step 1: Encode message and apply key-based encryption
    if message_bit == 1:
        qc.x(0)
    if key_bit == 1:
        qc.z(0)

    # Step 2: Create Bell pair between qubits 1 and 2
    qc.h(1)
    qc.cx(1, 2)

    # Step 3: Bell-state measurement on qubit 0 and 1
    qc.cx(0, 1)
    qc.h(0)
    qc.measure([0, 1], [0, 1])  # Classical results sent to Bob

    # Step 4: Bob applies conditional corrections on qubit 2
    qc.cx(1, 2)
    qc.cz(0, 2)
    qc.barrier()
    qc.measure(2, 2)  # Bob measures final qubit

    compiled = transpile(qc, simulator)
    result = simulator.run(compiled, shots=1).result()
    counts = result.get_counts()
    key = list(counts.keys())[0].replace(' ', '')
    final_bit = int(key[2])  # Qubit 2's measurement result
    return final_bit

# Main simulation driver
def main():
    print("\n--- Quantum Secure Messaging App ---\n")

    # Phase 1: Key generation using BB84 with eavesdropping
    print("[1] Key Distribution Phase (BB84) with Fake Eve...")
    alice_key, bob_key, matched_indices, eve_actions = generate_bb84_key(n_bits, eve_present=True)
    error_rate = simulate_eve_detection(alice_key, bob_key)
    print(f"\nKey agreement after sifting: {len(alice_key)} bits")
    print(f"Eavesdropping error rate estimate: {100*error_rate:.1f}%")

    print("\nEve's activity log (None means no Eve):")
    print([eve_actions[i] for i in matched_indices])

    # Warn user if error rate suggests eavesdropping
    if error_rate > 0.2:
        print("⚠️  Warning: Possible eavesdropper detected!")
        proceed = input("Do you want to proceed anyway for demo purposes? (y/n): ").strip().lower()
        if proceed != 'y':
            print("Aborting communication.")
            return

    # Phase 2: Send encrypted message using quantum teleportation
    print("\n[2] Secure Message Transmission (Quantum Teleportation)...")
    message = np.random.randint(2, size=len(alice_key))  # Random message bits
    encrypted_message = []
    for i, (bit, key) in enumerate(zip(message, alice_key)):
        print(f"  Teleporting bit {i+1}: message={bit}, key={key}")
        encrypted_bit = quantum_teleportation(bit, key)
        print(f"    Received: {encrypted_bit}\n")
        encrypted_message.append(encrypted_bit)
        time.sleep(0.2)  # Delay for pacing

    print(f"Original message:       {message}")
    print(f"Encrypted/teleported:   {encrypted_message}")
    print("Secure transmission complete.")

# Start the simulation
if __name__ == "__main__":
    main()


#### Start Your Own Project Below

In [ ]:
# Start Project

### 🌌 Quantum Sensing for Gravitational Waves  

**Concepts:** Ramsey Interferometry · Quantum Phase Sensitivity

**Learn:**
- How quantum sensors detect tiny spacetime changes.  
- The interference pattern caused by gravitational waves.

**Explore:**  
Simulate a Ramsey interferometer and identify gravitational effects from periodic phase shifts.


In [ ]:
# Quantum Sensing for Gravitational Waves via Ramsey Interferometry
# Simulates ultra-sensitive phase shifts due to gravitational wave effects

import numpy as np  # For numerical operations and phase generation
import matplotlib.pyplot as plt  # For plotting results
from qiskit import QuantumCircuit, transpile  # For building and compiling quantum circuits
from qiskit_aer import Aer  # Qiskit’s quantum simulator backend
import time  # For pacing output display

# Simulation settings
shots = 1024  # Number of repeated measurements per phase
phases = np.linspace(0, np.pi, 50)  # Range of phase shifts to simulate (0 to π)

# Ramsey interferometry function: models how a qubit responds to a phase shift
def ramsey_interference(phase):
    qc = QuantumCircuit(1, 1)  # One-qubit circuit with one classical bit
    qc.h(0)              # Step 1: Create a superposition using Hadamard gate
    qc.rz(phase, 0)      # Step 2: Apply a phase shift simulating gravitational wave effect
    qc.h(0)              # Step 3: Apply Hadamard again to interfere the paths
    qc.measure(0, 0)     # Step 4: Measure the result in the computational basis
    return qc

# Initialize simulator and storage
simulator = Aer.get_backend("aer_simulator")
prob_0_list = []  # Store probability of measuring |0⟩ for each phase

# Begin simulation
print("\n🌌 Simulating Ramsey Interferometer Response to Gravitational Waves...")
time.sleep(1)

# Loop over each phase value and simulate its effect
for i, phi in enumerate(phases):
    print(f"  Measuring phase shift {i+1}/{len(phases)}: {phi:.2f} radians", end="\r")
    circuit = ramsey_interference(phi)  # Build circuit with this phase
    compiled = transpile(circuit, simulator)  # Compile it for the simulator
    result = simulator.run(compiled, shots=shots).result()  # Run simulation
    counts = result.get_counts()  # Get measurement results
    prob_0 = counts.get('0', 0) / shots  # Calculate probability of measuring '0'
    prob_0_list.append(prob_0)  # Store it
    time.sleep(0.05)  # Small delay for smoother output

print("\n✅ Simulation complete!\n")

# Plot the interference pattern
plt.figure(figsize=(8, 5))
plt.plot(phases, prob_0_list, label='P(0)', color='navy')  # Probability curve
plt.xlabel('Phase Shift (rad)')
plt.ylabel('Probability of Measuring 0')
plt.title('🌀 Ramsey Interference from Gravitational Phase Shift')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Display interactive interpretation block using HTML (works in Jupyter/Colab)
import IPython
from IPython.display import display, HTML

question_html = '''
<div style="border:2px solid #1E88E5; border-radius: 10px; padding: 15px; background-color: #e3f2fd; color: #0d47a1;">
  <h3 style="margin-top: 0; color: black;">🚀 Imagine you are a quantum sensor on a satellite</h3>
  <p><strong>Did you detect a pattern consistent with a gravitational wave?</strong></p>
  <p><em>Type 'yes' or 'no' below:</em></p>
</div>
'''
display(HTML(question_html))  # Render the question for user interaction

# Ask user to interpret the pattern
try:
    response = input("Your answer (yes/no): ").strip().lower()
    if response == 'yes':
        print("✅ Great! The periodic pattern suggests sensitivity to gravitational wave effects.")
    elif response == 'no':
        print("🔍 Keep exploring! Try changing the phase dynamics or increasing resolution.")
    else:
        print("ℹ️ Please enter either 'yes' or 'no'.")
except:
    print("⚠️ Input not available in this environment. Proceeding with interpretation.")

# Explanation of physical meaning
print("\n🔍 Each phase value represents a tiny shift in spacetime caused by a passing gravitational wave.")
print("💡 The oscillating curve shows how sensitive quantum states are to these shifts.")
print("🧪 This is how real experiments like LIGO or atomic clocks detect ultra-fine changes in the universe.")

# Student Challenge
print("\n🔭 Student Challenge:")
print("Can you modify the phase function to represent a burst-like gravitational wave?")
print("Try applying a Gaussian modulation or chirp signal to see how it alters the pattern! 🚀")


#### Start Your Own Project Below

In [ ]:
# Start Here

### 🛰️ Quantum-enhanced GPS and Clock Network Simulation  

**Concepts:** Entanglement · Phase Drift · Synchronization

**Learn:**
- How quantum clocks detect drift.  
- How synchronization events restore coherence.

**Explore:**  
Visualize clock desynchronization and understand the power of entanglement in future GPS systems.


In [ ]:
# Quantum-enhanced GPS or Clock Network Simulation
# Simulates entangled clocks detecting phase drift and time synchronization

import numpy as np  # For numerical operations and drift simulation
import matplotlib.pyplot as plt  # For plotting the results
from qiskit import QuantumCircuit, transpile  # For quantum circuit creation and compilation
from qiskit_aer import Aer  # Qiskit simulator
from IPython.display import display, HTML  # For rich HTML output in notebooks
import time  # Used to add pacing to the simulation

# Settings
shots = 1024  # Number of times each quantum circuit is run
phases = np.linspace(0, 2 * np.pi, 60)  # Simulated total phase drift range
phase_drift_rate = 0.1  # Rate at which unsynced clocks drift
sync_events = [20, 40]  # Time points when network synchronizes clocks

# Function to simulate an entangled clock pair with phase drift
def entangled_clock_network(drift):
    qc = QuantumCircuit(2, 2)  # Two qubits (two clocks), two classical bits for measurement

    # Step 1: Create an entangled Bell state between the clocks
    qc.h(0)
    qc.cx(0, 1)

    # Step 2: Simulate phase drift in one of the clocks
    qc.rz(drift, 1)  # Drift applied to qubit 1 only

    # Step 3: Ramsey interferometry (sensitive to phase differences)
    qc.h(0)
    qc.h(1)

    # Step 4: Measure both qubits
    qc.measure([0, 1], [0, 1])
    return qc

# Prepare simulator and data list
simulator = Aer.get_backend("aer_simulator")
prob_diff = []  # Track mismatch probabilities (i.e., clocks disagreeing)

# Start simulation
print("\n🛰️ Simulating Entangled Clock Network Synchronization...\n")
time.sleep(1)

# Loop through each phase value, simulating time progression
for t, phi in enumerate(phases):
    if t in sync_events:
        print(f"⏱️  Network time update at t = {t}! Resynchronizing clocks.")
        drift = 0  # Synchronization resets the phase drift
    else:
        drift = phase_drift_rate * t  # Drift accumulates over time

    qc = entangled_clock_network(drift)  # Build quantum circuit for this drift
    compiled = transpile(qc, simulator)  # Compile for simulator
    result = simulator.run(compiled, shots=shots).result()  # Execute circuit
    counts = result.get_counts()  # Get measurement outcomes

    # Calculate the probability that the qubit measurements disagree (i.e., clocks out of sync)
    prob_mismatch = sum([v for k, v in counts.items() if k[0] != k[1]]) / shots
    prob_diff.append(prob_mismatch)
    time.sleep(0.02)  # Small delay for user feedback

print("\n✅ Simulation Complete\n")

# Plot mismatch probability over time as a measure of desynchronization
plt.figure(figsize=(9, 5))
plt.plot(phases, prob_diff, color='darkorange', linewidth=2, label='P(mismatch)')
plt.axvline(phases[sync_events[0]], color='green', linestyle='--', label='Sync Event 1')
plt.axvline(phases[sync_events[1]], color='blue', linestyle='--', label='Sync Event 2')
plt.xlabel('Phase Drift Accumulation (rad)')
plt.ylabel('Probability of Clock Disagreement')
plt.title('🔄 Clock Network Drift and Synchronization')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Interactive reflection block for student engagement
question_html = '''
<div style="border:2px solid #3f51b5; border-radius: 10px; padding: 15px; background-color: #f0f8ff; color: #1a237e;">
  <h3 style="margin-top: 0; color: black;">🧭 Quantum Timekeeping Reflection</h3>
  <p><strong>Question:</strong> Why do you think the mismatch probability resets at specific points?</p>
  <p><em>Type 'sync' if you think it's due to network time synchronization, or 'drift' if due to decoherence.</em></p>
</div>
'''
display(HTML(question_html))  # Works only in notebook environments

# Input and feedback
try:
    response = input("Your answer (sync/drift): ").strip().lower()
    if response == 'sync':
        print("\n✅ Correct! Clock synchronization resets the phase drift and restores agreement.")
    elif response == 'drift':
        print("\n❌ Actually, decoherence wasn't modeled here. The reset was due to synchronization.")
    else:
        print("\nℹ️ Please enter either 'sync' or 'drift'.")
except:
    print("\n⚠️ Input not available. Please consider this reflection offline.")

# Educational wrap-up
print("\n🛰️ This simulation shows how entangled qubits can be used to detect and correct drift in quantum clock networks.")
print("Future quantum-enhanced GPS systems could use similar techniques to achieve unprecedented precision.")


#### Start Your Own Project Below

In [ ]:
# Start Here

### 📡 Quantum Radar vs Classical Radar  

**Concepts:** Entangled Detection · Noise Comparison · Performance Metrics

**Learn:**
- How quantum radar uses entanglement to improve detection.  
- How false positives/negatives compare between radar types.

**Explore:**  
Run side-by-side radar simulations and interpret the detection outcome metrics.

In [ ]:
# Quantum Radar vs Classical Radar Simulation
# Compares quantum entangled detection with classical probabilistic radar under noise

import numpy as np  # For randomness and numerical operations
import matplotlib.pyplot as plt  # For plotting results
from qiskit import QuantumCircuit, transpile  # Qiskit tools for building quantum circuits
from qiskit_aer import Aer  # Quantum simulator backend
from IPython.display import display, HTML  # For HTML display in Jupyter
import time  # For pacing output display

# Simulation settings
shots = 1024  # Number of quantum measurements per radar trial
runs = 50  # Number of total radar simulations
noise_level = 0.2  # Noise standard deviation in classical radar
signal_present_prob = 0.6  # Probability that a signal is actually present

# Quantum radar simulation using entanglement
def quantum_radar(simulate_signal):
    qc = QuantumCircuit(2, 2)  # Two entangled qubits and two classical bits

    qc.h(0)  # Create superposition on qubit 0
    qc.cx(0, 1)  # Entangle qubit 0 with qubit 1 (Bell state)

    if simulate_signal:
        qc.z(1)  # Apply phase flip if signal is present (signal interaction effect)

    qc.h(0)
    qc.h(1)
    qc.measure([0, 1], [0, 1])  # Measure both qubits
    return qc

# Classical radar simulated as a noisy probabilistic threshold detector
def classical_radar(simulate_signal):
    signal_strength = np.random.rand() + (0.5 if simulate_signal else 0)  # Stronger if signal present
    noise = np.random.normal(0, noise_level)  # Add Gaussian noise
    detection_score = signal_strength + noise
    return detection_score > 0.75  # Threshold for declaring a detection

# Initialize results and simulator
simulator = Aer.get_backend("aer_simulator")
quantum_results, classical_results = [], []
true_labels = []

print("\n📡 Running Quantum vs Classical Radar Simulation...")

# Run simulations
for i in range(runs):
    is_signal = np.random.rand() < signal_present_prob  # Decide if signal is present this run
    true_labels.append(is_signal)

    # Quantum radar detection
    qc = quantum_radar(is_signal)
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled, shots=shots).result()
    counts = result.get_counts()

    # Determine if the entangled qubits' results disagree (mismatch = signal)
    prob_mismatch = sum(v for k, v in counts.items() if k[0] != k[1]) / shots
    quantum_detected = prob_mismatch > 0.1  # Arbitrary detection threshold
    quantum_results.append(quantum_detected)

    # Classical radar detection
    classical_detected = classical_radar(is_signal)
    classical_results.append(classical_detected)

    time.sleep(0.02)  # Small delay for visualization pacing

# Calculate performance metrics
def calculate_metrics(true_vals, predicted):
    tp = sum(t and p for t, p in zip(true_vals, predicted))  # True positives
    tn = sum(not t and not p for t, p in zip(true_vals, predicted))  # True negatives
    fp = sum(not t and p for t, p in zip(true_vals, predicted))  # False positives
    fn = sum(t and not p for t, p in zip(true_vals, predicted))  # False negatives
    return tp, tn, fp, fn

# Get metrics for both systems
q_tp, q_tn, q_fp, q_fn = calculate_metrics(true_labels, quantum_results)
c_tp, c_tn, c_fp, c_fn = calculate_metrics(true_labels, classical_results)

# Visualization: Detection outcome counts
x_labels = ["True Pos", "True Neg", "False Pos", "False Neg"]
q_vals = [q_tp, q_tn, q_fp, q_fn]
c_vals = [c_tp, c_tn, c_fp, c_fn]
x = np.arange(len(x_labels))

plt.figure(figsize=(10, 6))
plt.bar(x - 0.2, q_vals, width=0.4, label='Quantum Radar', color='purple')  # Quantum bars
plt.bar(x + 0.2, c_vals, width=0.4, label='Classical Radar', color='gray')  # Classical bars
plt.xticks(x, x_labels)
plt.ylabel("Counts")
plt.title("🔍 Quantum vs Classical Radar Detection Outcomes")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Interactive analysis: Ask students to compare false positives
question_html = '''
<div style="border:2px solid #2196F3; border-radius: 10px; padding: 15px; background-color: #f5faff; color: #0d47a1;">
  <h3 style="margin-top: 0; color: black;">🛰️ Radar System Challenge</h3>
  <p><strong>Question:</strong> Which radar system had fewer false positives?</p>
  <p><em>Type 'quantum' or 'classical' below:</em></p>
</div>
'''
display(HTML(question_html))

# Handle user input for engagement
try:
    response = input("Your answer (quantum/classical): ").strip().lower()
    if response == 'quantum' and q_fp < c_fp:
        print("\n✅ Correct! Quantum radar showed fewer false alarms under noisy conditions.")
    elif response == 'classical' and c_fp < q_fp:
        print("\n✅ Correct! Classical radar handled this noise scenario better in false alarms.")
    else:
        print("\n🤔 Not quite. Recheck the plot and compare false positive bars.")
except:
    print("\n⚠️ Input not supported in this environment. Reflect on the comparison visually.")

# Educational wrap-up
print("\n📡 Summary: Quantum radar leverages entanglement to detect subtle differences, often reducing false positives.")
print("Classical radar is simpler but more sensitive to noise fluctuations, leading to more false alarms in some cases.")


#### Start Your Own Project Below

In [ ]:
# Start Here

# 🧩 Interactive Quantum Games

### These projects are gamified and exploratory, designed for engagement and intuitive understanding of quantum principles through decision-making, scoring, and circuit visualization.

### 🎮 Spy vs Spy — Interactive QKD Simulation Game  

**Concepts:** BB84 QKD · Eavesdropping · Circuit Fidelity

**Learn:**
- How Alice and Bob exchange keys.  
- The impact of Eve's presence.  
- Circuit behavior at different difficulty levels.

**Play:**  
Choose your mission level, survive Eve's attacks, and rack up your fidelity score.


In [ ]:
# Quantum Game: Spy vs Spy — Interactive QKD Simulation Game with Circuit Visuals

import numpy as np  # For generating random bits and bases
import matplotlib.pyplot as plt  # For plotting key comparisons
from qiskit import QuantumCircuit, transpile  # For building and compiling quantum circuits
from qiskit_aer import Aer  # Qiskit simulator backend
from qiskit.visualization import circuit_drawer  # For drawing quantum circuits
from IPython.display import display, HTML  # For HTML display in Jupyter
import random  # For consistent randomness
import time  # For adding delays and pacing

# Game settings
shots = 1  # Each qubit is only measured once, as in real QKD
np.random.seed(42)
random.seed(42)

# Initialize game state
score = 0  # Player's score across rounds
level = 1  # Game level

# Simulates one round of quantum key distribution (QKD)
def play_round(num_bits=10, eve_active=True):
    # Generate random bits and basis choices
    alice_bits = np.random.randint(2, size=num_bits)  # Alice's bit values (0 or 1)
    alice_bases = np.random.randint(2, size=num_bits)  # Alice's bases (0=Z, 1=X)
    bob_bases = np.random.randint(2, size=num_bits)  # Bob's bases
    eve_bases = np.random.randint(2, size=num_bits)  # Eve's random basis choices

    bob_results = []  # Store Bob's measurement outcomes
    circuits = []  # Store the circuits for visualization

    simulator = Aer.get_backend('aer_simulator')

    # Loop through each qubit (bit position)
    for i in range(num_bits):
        qc = QuantumCircuit(1, 1)

        # Alice encodes her bit
        if alice_bits[i] == 1:
            qc.x(0)
        if alice_bases[i] == 1:
            qc.h(0)

        # Eve intercepts and measures (if active)
        if eve_active:
            eve_basis = eve_bases[i]
            if eve_basis == 1:
                qc.h(0)
            qc.measure(0, 0)
            compiled = transpile(qc, simulator)
            result = simulator.run(compiled, shots=1, memory=True).result()
            intercepted = int(result.get_memory()[0])

            # Eve prepares a new qubit based on her measurement
            qc = QuantumCircuit(1, 1)
            if intercepted == 1:
                qc.x(0)
            if eve_basis == 1:
                qc.h(0)

        # Bob applies his basis and measures
        if bob_bases[i] == 1:
            qc.h(0)
        qc.measure(0, 0)

        compiled = transpile(qc, simulator)
        result = simulator.run(compiled, shots=1, memory=True).result()
        bob_results.append(int(result.get_memory()[0]))

        # Save the circuit for later display
        circuits.append(qc.copy())

    # Compare bases to create sifted keys
    match_indices = [i for i in range(num_bits) if alice_bases[i] == bob_bases[i]]
    alice_key = [alice_bits[i] for i in match_indices]
    bob_key = [bob_results[i] for i in match_indices]

    # Measure fidelity: how many bits match?
    mismatches = sum(1 for a, b in zip(alice_key, bob_key) if a != b)
    fidelity = 1 - mismatches / len(match_indices) if match_indices else 0

    return fidelity, alice_key, bob_key, circuits

# Show visual comparison of Alice's and Bob's keys
def show_keys(alice_key, bob_key):
    x = np.arange(len(alice_key))
    plt.figure(figsize=(10, 4))
    plt.plot(x, alice_key, 'go-', label='Alice Key')
    plt.plot(x, bob_key, 'ro--', label='Bob Key')
    plt.title('🔐 Key Match Visual')
    plt.xlabel('Bit Index')
    plt.ylabel('Bit Value')
    plt.ylim(-0.2, 1.2)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Show sample quantum circuits from the round
def show_circuits(circuits):
    print("\n🧩 Sample Quantum Circuits from This Round:")
    for i, qc in enumerate(circuits[:3]):  # Show only first 3 for clarity
        print(f"\n🌀 Circuit {i+1}:")
        display(circuit_drawer(qc, output='mpl'))

# Start game message
print("\n🎮 Welcome to Spy vs Spy: Quantum Edition!")
print("Uncover secret keys. Avoid interception. Outsmart Eve.")
time.sleep(1)

# Main game loop: 3 levels
for stage in range(3):
    print(f"\n🔓 Level {level}: Choose your mission difficulty!")
    print("1: Easy (no Eve)")
    print("2: Medium (some Eve)")
    print("3: Hard (Eve always present)")

    choice = input("Select your mission (1/2/3): ").strip()
    eve_presence = True if choice == '2' or choice == '3' else False

    # Play one round of the game
    fidelity, alice_key, bob_key, circuits = play_round(num_bits=10 + level*5, eve_active=eve_presence)
    print(f"\n🧠 Fidelity this round: {fidelity:.2f}")

    # Scoring based on fidelity
    if fidelity > 0.9:
        score += 100
        print("✅ Mission Success! Your key remained secret.")
    elif fidelity > 0.6:
        score += 50
        print("⚠️ Partial success. Eve might’ve seen some bits.")
    else:
        print("❌ Key compromised. Eve intercepted too much!")

    # Visual feedback
    show_keys(alice_key, bob_key)
    show_circuits(circuits)

    level += 1
    time.sleep(2)

# Final score and summary
print("\n🏁 Game Over!")
print(f"🎯 Final Score: {score}")
if score >= 250:
    print("🥇 Quantum Mastermind! You kept your secrets safe.")
elif score >= 150:
    print("🥈 Solid Agent. Just a few leaks.")
else:
    print("🥉 Training needed. Eve outsmarted you.")


#### Start Your Own Project Below

In [ ]:
# Start Here

### 🧪 Quantum Device Museum — Gamified Digital Exhibit  

**Concepts:** Atomic Clocks · Quantum Radar · QKD

**Learn:**
- How fundamental quantum devices work.  
- Phase estimation and detection.  
- Eavesdropping in QKD.

**Play:**  
Visit exhibits, make predictions, and see your final score based on performance across different challenges.

In [ ]:
# Quantum Device Museum — Gamified Digital Exhibit
# Interactive simulation of quantum technologies: atomic clocks, QKD, and quantum radar

import numpy as np  # For phase calculations
import matplotlib.pyplot as plt  # For visual feedback (not used directly here)
from qiskit import QuantumCircuit, transpile  # Quantum circuit construction and compilation
from qiskit_aer import Aer  # Qiskit’s simulator backend
from qiskit.visualization import circuit_drawer  # Draw quantum circuits as images
from IPython.display import display, Markdown  # For displaying formatted text in notebooks
import math  # For mathematical constants like pi
import time
import random  # For randomized game scenarios

score = 0  # Track user’s score across all exhibits

# -------- Exhibit 1: Atomic Clock Simulation --------
def atomic_clock():
    global score
    display(Markdown("## ⏰ Atomic Clock Challenge"))
    display(Markdown("You're calibrating a Ramsey interferometer to detect a phase shift of π/4. Can you guess the measurement outcome?"))

    # Build the Ramsey interferometer circuit
    qc = QuantumCircuit(1, 1)
    qc.h(0)  # Superposition
    qc.rz(math.pi / 4, 0)  # Simulated phase drift
    qc.h(0)  # Interference
    qc.measure(0, 0)

    # Run the circuit
    simulator = Aer.get_backend('aer_simulator')
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled, shots=1, memory=True).result()
    measured = result.get_memory()[0]  # Read measured bit (0 or 1)

    # Display circuit
    display(circuit_drawer(qc, output='mpl'))

    # Ask user to predict the measurement
    guess = input("Will the atom end in state 0 or 1? (enter 0 or 1): ").strip()
    if guess == measured:
        display(Markdown("✅ Correct! You understand quantum phase accumulation."))
        score += 10
    else:
        display(Markdown(f"❌ Nope! The atom was measured in state {measured}."))

# -------- Exhibit 2: Quantum Key Distribution Game --------
def qkd_network():
    global score
    display(Markdown("## 🔐 Quantum Key Distribution Game"))
    display(Markdown("You're Alice. Choose a basis (Z or X) to encode a bit. Eve might be listening!"))

    # Random bit and eavesdropping chance
    alice_bit = random.randint(0, 1)
    alice_basis = input("Choose a basis (Z or X): ").strip().upper()
    eve_present = random.choice([True, False])

    qc = QuantumCircuit(1, 1)

    # Alice encodes bit
    if alice_bit == 1:
        qc.x(0)
    if alice_basis == 'X':
        qc.h(0)

    # Eve may intercept and measure, disturbing the qubit
    if eve_present:
        eve_basis = random.choice(['Z', 'X'])
        if eve_basis == 'X':
            qc.h(0)
        qc.measure(0, 0)
        result = Aer.get_backend('aer_simulator').run(
            transpile(qc, Aer.get_backend('aer_simulator')), shots=1, memory=True).result()
        intercepted = int(result.get_memory()[0])
        qc = QuantumCircuit(1, 1)
        if intercepted == 1:
            qc.x(0)
        if eve_basis == 'X':
            qc.h(0)

    # Bob measures
    qc.measure(0, 0)
    display(circuit_drawer(qc, output='mpl'))
    result = Aer.get_backend('aer_simulator').run(
        transpile(qc, Aer.get_backend('aer_simulator')), shots=1, memory=True).result()
    bob_bit = int(result.get_memory()[0])
    display(Markdown(f"📡 Bob received: {bob_bit}"))

    # Feedback on eavesdropping
    if eve_present:
        display(Markdown("⚠️ Eve was eavesdropping!"))
    else:
        display(Markdown("✅ No eavesdropping detected."))

    # Result evaluation
    if bob_bit == alice_bit:
        score += 10
        display(Markdown("🎉 Secure key shared!"))
    else:
        display(Markdown("🔓 Key mismatch due to basis misalignment or eavesdropping."))

# -------- Exhibit 3: Quantum Radar Detection --------
def quantum_radar():
    global score
    display(Markdown("## 📡 Quantum Radar Detection"))
    display(Markdown("Can you detect the phase shift from a returning entangled signal?"))

    # Choose random phase shift (unknown to player)
    phase_shift = random.choice([0, np.pi/4, np.pi/2])

    # Quantum radar circuit using entangled Bell pair
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)  # Create entangled pair
    qc.rz(phase_shift, 1)  # Incoming signal applies unknown phase
    qc.cx(0, 1)
    qc.h(0)
    qc.measure([0, 1], [0, 1])

    display(circuit_drawer(qc, output='mpl'))
    guess = input("Guess the phase shift (0, pi/4, pi/2): ").strip()

    # Compare guess with actual hidden phase
    if guess == '0' and phase_shift == 0 or \
       guess == 'pi/4' and phase_shift == np.pi/4 or \
       guess == 'pi/2' and phase_shift == np.pi/2:
        display(Markdown("✅ Correct! Target phase detected."))
        score += 15
    else:
        display(Markdown(f"❌ Wrong. Actual shift was {round(phase_shift, 2)} radians."))

# -------- Run the Digital Museum Exhibits --------
display(Markdown("# 🧪 Quantum Device Museum"))
atomic_clock()
qkd_network()
quantum_radar()

# Final Score Display
display(Markdown("---"))
display(Markdown(f"## 🎯 Final Score: {score}"))
display(Markdown("Thanks for playing! Each quantum device helps shape the future of technology."))


#### Start Your Own Project Below

In [ ]:
# Start Here

### 🚨 Quantum Emergency Game: Save the Satellite!

**Concepts:** Quantum Teleportation · Ramsey Interferometry · Quantum Response Simulation

**Learn:**
- How to use quantum tools to resolve real-time satellite issues.  
- When to teleport, recalibrate, or wait.  
- The impact of your decisions.

**Play:**  
Respond to system alerts and try to recover a satellite's quantum systems across three rounds.

In [ ]:
# 🚨 Quantum Emergency Game: Save the Satellite!
# Interactive simulation using quantum teleportation, Ramsey interferometry, and decision-making under uncertainty

import numpy as np
import math
import random
from qiskit import QuantumCircuit, transpile  # Quantum circuit construction
from qiskit_aer import Aer  # Quantum simulator backend
from qiskit.visualization import circuit_drawer  # To visualize circuits
from IPython.display import display, Markdown  # For styled text in notebooks

# Initialize game state
score = 0
rounds = 3  # Total number of emergency events to respond to

# Function for each round
def play_round(round_num):
    global score
    display(Markdown(f"### 🧪 Round {round_num}: Satellite System Alert!"))

    # Simulate a random quantum error affecting the satellite
    error_type = random.choice(["timing drift", "sensor decoherence", "signal loss"])
    satellite_state = random.choice([0, 1])  # True state of the satellite system
    display(Markdown(f"⚠️ Detected issue: **{error_type.upper()}**"))

    # Present options to the player
    display(Markdown("Choose your response:"))
    display(Markdown("1️⃣ Teleport qubit state from satellite"))
    display(Markdown("2️⃣ Attempt recalibration via Ramsey interferometry"))
    display(Markdown("3️⃣ Wait and observe another cycle"))

    valid_choice = False
    while not valid_choice:
        try:
            choice = int(input("Enter 1, 2, or 3: "))
            if choice in [1, 2, 3]:
                valid_choice = True
        except:
            continue

    # --- Option 1: Quantum Teleportation ---
    if choice == 1:
        display(Markdown("🛰️ Executing quantum teleportation protocol..."))
        qc_sender = QuantumCircuit(3, 2)

        # Step 1: Encode satellite state in qubit 0
        if satellite_state == 1:
            qc_sender.x(0)

        # Step 2: Create Bell pair (entanglement) between qubit 1 and 2
        qc_sender.h(1)
        qc_sender.cx(1, 2)

        # Step 3: Bell measurement between qubit 0 and 1
        qc_sender.cx(0, 1)
        qc_sender.h(0)
        qc_sender.measure([0, 1], [0, 1])

        # Step 4: Run the sender circuit
        simulator = Aer.get_backend('aer_simulator')
        compiled = transpile(qc_sender, simulator)
        result = simulator.run(compiled, shots=1, memory=True).result()
        bits = result.get_memory()[0]
        m0, m1 = int(bits[1]), int(bits[0])  # Measurement results

        # Step 5: Receiver applies correction based on classical info
        qc_receiver = QuantumCircuit(1, 1)
        if m1 == 1:
            qc_receiver.x(0)
        if m0 == 1:
            qc_receiver.z(0)
        qc_receiver.measure(0, 0)

        # Step 6: Measure the final state after corrections
        result2 = simulator.run(transpile(qc_receiver, simulator), shots=1, memory=True).result()
        teleported = int(result2.get_memory()[0])

        # Display circuit and outcome
        display(circuit_drawer(qc_sender, output='mpl'))
        display(Markdown(f"📡 Received state: **{teleported}** (Expected: {satellite_state})"))

        # Scoring
        if teleported == satellite_state:
            display(Markdown("✅ Teleportation successful! System back online."))
            score += 20
        else:
            display(Markdown("❌ Teleportation failed. State mismatch detected."))

    # --- Option 2: Ramsey Interferometry Calibration ---
    elif choice == 2:
        display(Markdown("🔬 Running Ramsey interferometry..."))
        qc = QuantumCircuit(1, 1)

        # Step 1: Prepare superposition
        qc.h(0)

        # Step 2: Apply simulated phase shift based on actual state
        phase = math.pi / 4 if satellite_state == 1 else 0
        qc.rz(phase, 0)

        # Step 3: Interfere and measure
        qc.h(0)
        qc.measure(0, 0)

        # Execute circuit
        simulator = Aer.get_backend('aer_simulator')
        compiled = transpile(qc, simulator)
        result = simulator.run(compiled, shots=1, memory=True).result()
        measured = int(result.get_memory()[0])

        # Display and score
        display(circuit_drawer(qc, output='mpl'))
        display(Markdown(f"🧭 Ramsey output: {measured} (Expected: {satellite_state})"))

        if measured == satellite_state:
            display(Markdown("✅ Recalibration successful! Timing corrected."))
            score += 15
        else:
            display(Markdown("❌ Recalibration error. More tuning needed."))

    # --- Option 3: Passive Monitoring ---
    elif choice == 3:
        outcome = random.choice(["improved", "worsened"])
        if outcome == "improved":
            display(Markdown("🕒 Time passed... sensors self-corrected!"))
            score += 10
        else:
            display(Markdown("🕒 Time passed... signal degradation worsened!"))
            score -= 5

# Master game controller
def quantum_emergency_game():
    display(Markdown("# 🚨 Quantum Emergency Game: Save the Satellite!"))
    display(Markdown("You're the quantum systems officer. A satellite's quantum sensor array is failing. Can you make the right decisions to recover functionality?"))

    # Play multiple rounds
    for r in range(1, rounds + 1):
        play_round(r)

    # Final outcome
    display(Markdown("---"))
    display(Markdown(f"## 🏁 Final Score: {score} / {rounds * 20}"))
    if score >= rounds * 18:
        display(Markdown("🌟 Stellar response! Full system restored."))
    elif score >= rounds * 10:
        display(Markdown("👍 Good recovery. Partial sync achieved."))
    else:
        display(Markdown("⚠️ Mission critical. Satellite still at risk!"))

# Run the game
quantum_emergency_game()


#### Start Your Own Project Below

In [ ]:
# Start Here